## COLAB NOTEBOOK FOR THE GPT-3.5-TURBO RESULTS EVALUATIONS

(see the flan-t5 or azure notebooks for a more ordered/detailed execution, which is almost identical)

In [1]:
!pip install --upgrade pip
!pip install unbabel-comet
!pip install ast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.6/81.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 73.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 118.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1

In [3]:
# Retrieve the python dictionary
import ast
data = []
with open("gpt_dict.txt", "r", encoding='utf-8') as file:
    for line in file:
        dictionary_str = line.strip()
        entry = ast.literal_eval(dictionary_str)
        data.append(entry)

### COMET-22 METRIC

In [6]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(model_path)
model_output = model.predict(data, batch_size=8, gpus=1)
print(model_output)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 7453/7453 [34:59<00:00,  3.55it/s]


Prediction([('scores', [0.9392632842063904, 0.8450779318809509, 0.9916894435882568, 0.8713397979736328, 0.8015279173851013, 0.9097328782081604, 0.8925313353538513, 0.9376969933509827, 0.9055324196815491, 0.8512045741081238, 0.7877049446105957, 0.73617023229599, 0.8963430523872375, 0.7888355851173401, 0.9376969933509827, 0.8230791687965393, 0.8512045741081238, 0.7877049446105957, 0.8290402293205261, 0.8505284190177917, 0.8648724555969238, 0.9200635552406311, 0.8718287944793701, 0.8591805100440979, 0.8957769274711609, 0.8479750752449036, 0.9282416701316833, 0.8401946425437927, 0.8705628514289856, 0.9242767691612244, 0.8651613593101501, 0.8692185282707214, 0.8507348895072937, 0.7834272980690002, 0.9054032564163208, 0.8591268658638, 0.8540322184562683, 0.7887175679206848, 0.8947474360466003, 0.8561421036720276, 0.7671187520027161, 0.7699882388114929, 0.9511843323707581, 0.8620666861534119, 0.8687732815742493, 0.8088533282279968, 0.8030879497528076, 0.8335933685302734, 0.7662874460220337, 0

In [45]:
print(model_output[1])

0.8197010923958704


In [34]:
gpt_score = model_output[0]
gpt_score.append(model_output[1])

In [46]:
print(gpt_score[-1])

0.8197010923958704


In [36]:
with open("gpt_comet22.txt", "w") as file:
    for number in gpt_score:
        file.write(str(number) + "\n")

#with open("gpt_comet22.txt", "r") as file:
#    numbers = [int(line.strip()) for line in file.readlines()]

## BERTScore METRIC

In [48]:
!pip install bert-score git+https://github.com/huggingface/evaluate@af3c30561d840b83e54fc5f7150ea58046d6af69

  Cloning https://github.com/huggingface/evaluate (to revision af3c30561d840b83e54fc5f7150ea58046d6af69) to /tmp/pip-req-build-xdg2em7y
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/evaluate /tmp/pip-req-build-xdg2em7y
  Running command git rev-parse -q --verify 'sha^af3c30561d840b83e54fc5f7150ea58046d6af69'
  Running command git fetch -q https://github.com/huggingface/evaluate af3c30561d840b83e54fc5f7150ea58046d6af69
  Resolved https://github.com/huggingface/evaluate to commit af3c30561d840b83e54fc5f7150ea58046d6af69
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.2 MB/s eta 0:00:00
  Created wheel for evaluate: filename=evaluate-0.4.1.dev0-py3-none-any

In [51]:
predictions = [dictio['mt'] for dictio in data]
references = [dictio['ref'] for dictio in data]

In [55]:
from evaluate import load
bertscore = load("bertscore")
bert_output = bertscore.compute(predictions=predictions, references=references, lang="en")
print(bert_output)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [60]:
print('Precision: ', sum(bert_output['precision'])/len(bert_output['precision']))
print('Recall: ', sum(bert_output['recall'])/len(bert_output['recall']))
print('F1: ', sum(bert_output['f1'])/len(bert_output['f1']))

Precision:  0.9454668603534808
Recall:  0.9433467266570319
F1:  0.944326661075508


In [61]:
gpt_bert_score = bert_output['precision']
gpt_bert_score.append(sum(gpt_bert_score)/len(gpt_bert_score))
print(gpt_bert_score[-1])

gpt_bert_recall = bert_output['recall']
gpt_bert_recall.append(sum(gpt_bert_recall)/len(gpt_bert_recall))
print(gpt_bert_recall[-1])

gpt_bert_f1 = bert_output['f1']
gpt_bert_f1.append(sum(gpt_bert_f1)/len(gpt_bert_f1))
print(gpt_bert_f1[-1])

0.9454668603534808
0.9433467266570319
0.944326661075508


In [62]:
with open("gpt_bert_score.txt", "w") as file:
    for number in gpt_bert_score:
        file.write(str(number) + "\n")

with open("gpt_bert_recall.txt", "w") as file:
    for number in gpt_bert_recall:
        file.write(str(number) + "\n")

with open("gpt_bert_f1.txt", "w") as file:
    for number in gpt_bert_f1:
        file.write(str(number) + "\n")

### BLEURT METRIC

In [66]:
!pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-cds67mzi
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-cds67mzi
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456764 sha256=633e1c221b38f7dd1872aa7f7fe3c66ca5d6e2777e6f94a1f4461c0cf06b4bfa
  Stored in directory: /tmp/pip-ephem-wheel-cache-jbuyblv5/wheels/64/f4/2c/509a6c31b8ebde891a81029fd94f199b1b92f0e7cfc20d417a
Successfully built BLEURT


In [67]:
bleurt = load("bleurt", module_type="metric", checkpoint="BLEURT-20")
bleurt_output = bleurt.compute(predictions=predictions, references=references)
print(type(bleurt_output))

<class 'dict'>


In [70]:
gpt_bleurt_score = bleurt_output['scores']
means = sum(gpt_bleurt_score)/len(gpt_bleurt_score)
gpt_bleurt_score.append(means)
print(means)    # from -infty to +infty

0.31758796461002764


In [71]:
with open("gpt_bleurt.txt", "w") as file:
    for number in gpt_bleurt_score:
        file.write(str(number) + "\n")

### METEOR METRIC

In [74]:
!pip install nltk
!pip install git+https://github.com/huggingface/evaluate@af3c30561d840b83e54fc5f7150ea58046d6af69

  Cloning https://github.com/huggingface/evaluate (to revision af3c30561d840b83e54fc5f7150ea58046d6af69) to /tmp/pip-req-build-kutjkuuk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/evaluate /tmp/pip-req-build-kutjkuuk
  Running command git rev-parse -q --verify 'sha^af3c30561d840b83e54fc5f7150ea58046d6af69'
  Running command git fetch -q https://github.com/huggingface/evaluate af3c30561d840b83e54fc5f7150ea58046d6af69
  Resolved https://github.com/huggingface/evaluate to commit af3c30561d840b83e54fc5f7150ea58046d6af69
  Preparing metadata (setup.py) ... done


In [76]:
import evaluate

meteor = evaluate.load('meteor')
meteor_output = meteor.compute(predictions=predictions, references=references)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [80]:
gpt_meteor_score = meteor_output['meteor']
print(gpt_meteor_score)
#means = sum(gpt_bleurt_score)/len(gpt_bleurt_score)
#gpt_bleurt_score.append(means)


0.6372951527872919


In [84]:
from nltk.tokenize import word_tokenize
import evaluate

# Second version (PRETOKENIZED)

# Tokenize predictions
tokenized_predictions = [word_tokenize(pred) for pred in predictions]
# Join tokenized predictions
joined_predictions = [" ".join(tokens) for tokens in tokenized_predictions]

# Tokenize references
tokenized_references = [word_tokenize(ref) for ref in references]
# Join tokenized references
joined_references = [" ".join(tokens) for tokens in tokenized_references]

#meteor = evaluate.load('meteor')
meteor_output_tokenizedstrat = meteor.compute(predictions=joined_predictions, references=joined_references)

In [86]:
gpt_meteor_scorev2 = meteor_output_tokenizedstrat['meteor']
print(gpt_meteor_scorev2)

0.6373041950167946


In [87]:
with open("gpt_meteor.txt", "w") as file:
    for number in [gpt_meteor_scorev2]:
        file.write(str(number) + "\n")

### CHRF(++)

In [89]:
import evaluate

stacked_references = [[ref] for ref in references]
chrf = evaluate.load("chrf")
chrf_output = chrf.compute(predictions=predictions, references=stacked_references)
print(chrf_output) #from 0 to 100

{'score': 61.81223205018635, 'char_order': 6, 'word_order': 0, 'beta': 2}


CHRF++

In [90]:
chrf_output_pp = chrf.compute(predictions=predictions, references=stacked_references, word_order=2)
print(chrf_output_pp) #from 0 to 100

{'score': 59.02489057301943, 'char_order': 6, 'word_order': 2, 'beta': 2}


In [91]:
# Save the dictionaries to text files
import json

with open('gpt_chrf.txt', 'w') as file:
    json.dump(chrf_output, file)

with open('gpt_chrf_pp.txt', 'w') as file:
    json.dump(chrf_output_pp, file)

# Load the dictionaries from the text files
#with open('gpt_chrf.txt', 'r') as file:
#    loaded_data = json.load(file)
# ...

### BLEU

In [92]:
bleu = evaluate.load("bleu")
bleu_output = bleu.compute(predictions=predictions, references=stacked_references)
print(bleu_output)

{'bleu': 0.33870225698427303, 'precisions': [0.6389419053178051, 0.3949220526749424, 0.2705692168041858, 0.19276190952878494], 'brevity_penalty': 1.0, 'length_ratio': 1.0098655561645886, 'translation_length': 1295196, 'reference_length': 1282543}


In [94]:
import nltk
from nltk.tokenize import word_tokenize

# Initialize NLTK's word tokenizer
nltk.download('punkt')  # Download necessary resources for tokenization

# Tokenize the sentences into words and lowercase them
ntokenized_predictions = [word_tokenize(sentence.lower()) for sentence in predictions]
ntokenized_references = [word_tokenize(sentence.lower()) for sentence in references]

# Join the tokens of each sentence back into a string
njoined_predictions = [" ".join(tokens) for tokens in ntokenized_predictions]
njoined_references = [" ".join(tokens) for tokens in ntokenized_references]

nstacked_references = [[ref] for ref in njoined_references]

bleu_outputv2 = bleu.compute(predictions=njoined_predictions, references=nstacked_references)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [96]:
print(bleu_outputv2)

{'bleu': 0.35997868313910064, 'precisions': [0.6645889368765658, 0.4191071178326764, 0.28964898514156606, 0.20814068538074287], 'brevity_penalty': 1.0, 'length_ratio': 1.007139899460928, 'translation_length': 1301260, 'reference_length': 1292035}
59618 59618


In [97]:
with open('gpt_bleu.txt', 'w') as file:
    json.dump(bleu_outputv2, file)

In [98]:
bleu_outputv3 = bleu.compute(predictions=njoined_predictions, references=nstacked_references, max_order=2)
print(bleu_outputv3)


{'bleu': 0.5277631607812543, 'precisions': [0.6645889368765658, 0.4191071178326764], 'brevity_penalty': 1.0, 'length_ratio': 1.007139899460928, 'translation_length': 1301260, 'reference_length': 1292035}


A more heavy tokenizer would yield better results but what's the point?? The problem is in our peculiar corpus, the translation is good as we can see with the neural-based metrics.
Also, of course, the results with this little max_order are way better but such little context makes no sense.